# K8s Python API - Creating Deploytment and Service

## Install python api package

In [1]:
#!pip install kubernetes

## Imports

In [2]:
from kubernetes import client, config, utils
import yaml

## Yaml Templates for k8s objects

In [3]:
deployment_yaml = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: '**place-holder**'
spec:
  selector:
    matchLabels:
      run: '**place-holder**'
  replicas: 2
  template:
    metadata:
      labels:
        run: '**place-holder**'
    spec:
      containers:
      - name: my-nginx
        image: nginx
        ports:
        - containerPort: 80
"""

service_yaml = """
apiVersion: v1
kind: Service
metadata:
  name: '**place-holder**'
#  labels:
#    run: '**place-holder**'
spec:
  ports:
  - port: 80
    protocol: TCP
  selector:
    run: '**place-holder**'
"""


## setup for for accessing k8s

In [4]:
# get in-cluster config
config.load_incluster_config()

# Client API Interface
api_client = client.ApiClient()

# V1 Core API Interface
v1_api = client.CoreV1Api()

# V1 Apps API Interface
v1_app_api = client.AppsV1Api()

## Create deployment

In [5]:
# convert yaml template to dictionary
deployment = yaml.load(deployment_yaml, Loader=yaml.SafeLoader)

# update name
deployment['metadata']['name'] = 'jmt-nginx'
deployment['spec']['selector']['matchLabels']['run'] = 'jmt-nginx'
deployment['spec']['template']['metadata']['labels']['run'] =  'jmt-nginx'
                                              

# Pass to api client to create deployment object
#deployment_objects = utils.create_from_dict(api_client, deployment, namespace='kubeflow-user')

In [6]:
# convert template to dictionary
service = yaml.load(service_yaml, Loader=yaml.SafeLoader)

# modify for specific service deployment
service['metadata']['name'] = 'jmt-nginx'
service['metadata']['labels'] = {'run': 'jmt-nginx'}
service['spec']['selector']['run'] = 'jmt-nginx'

# pass to api client to create service object
# service_objects = utils.create_from_dict(api_client, service, namespace='kubeflow-user')

In [7]:
# setup list of k8s resources to instanstiate
k8s_resource_list = {
    'apiVersion': 'v1',
    'kind': 'List',
    'items': [deployment, service]
}

# instantiate the k8s resources
k8s_objects = utils.create_from_dict(api_client, k8s_resource_list, namespace='kubeflow-user')

## show k8s objects

In [8]:
for o in k8s_objects:
    print(o.metadata.name, o.kind)

jmt-nginx Deployment
jmt-nginx Service


In [10]:
!kubectl get deploy

NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
ml-pipeline-visualizationserver   1/1     1            1           18h
ml-pipeline-ui-artifact           1/1     1            1           18h
jmt-nginx                         2/2     2            2           16s


In [11]:
!kubectl describe deploy jmt-nginx

Name:                   jmt-nginx
Namespace:              kubeflow-user
CreationTimestamp:      Sun, 19 Dec 2021 17:22:48 +0000
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               run=jmt-nginx
Replicas:               2 desired | 2 updated | 2 total | 2 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  run=jmt-nginx
  Containers:
   my-nginx:
    Image:        nginx
    Port:         80/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
  Progressing    True    NewReplicaSetAvailable
OldReplicaSets:  <none>
NewReplicaSet:   jmt-nginx-5bb4fb5ff4 (2/2 replicas created)
Events:
  Type    Reason             Age   From             

In [12]:
!kubectl get pod -l run=jmt-nginx

NAME                         READY   STATUS    RESTARTS   AGE
jmt-nginx-5bb4fb5ff4-7rb8f   2/2     Running   0          17s
jmt-nginx-5bb4fb5ff4-fxggt   2/2     Running   0          18s


In [13]:
# find first pod of the deployment
ret = v1_api.list_namespaced_pod('kubeflow-user')
for i in ret.items:
    if i.metadata.labels.get('run', None) == 'jmt-nginx':
        pod_to_list = i.metadata.name
        break

In [14]:
!kubectl describe pod $pod_to_list

Name:         jmt-nginx-5bb4fb5ff4-7rb8f
Namespace:    kubeflow-user
Priority:     0
Node:         k3d-kubeflow-server-0/172.19.0.3
Start Time:   Sun, 19 Dec 2021 17:22:51 +0000
Labels:       istio.io/rev=default
              pod-template-hash=5bb4fb5ff4
              run=jmt-nginx
              security.istio.io/tlsMode=istio
              service.istio.io/canonical-name=jmt-nginx
              service.istio.io/canonical-revision=latest
Annotations:  kubectl.kubernetes.io/default-logs-container: my-nginx
              prometheus.io/path: /stats/prometheus
              prometheus.io/port: 15020
              prometheus.io/scrape: true
              sidecar.istio.io/status:
                {"initContainers":["istio-init"],"containers":["istio-proxy"],"volumes":["istio-envoy","istio-data","istio-podinfo","istio-token","istiod-...
Status:       Running
IP:           10.42.0.133
IPs:
  IP:           10.42.0.133
Controlled By:  ReplicaSet/jmt-nginx-5bb4fb5ff4
Init Containers:
  istio-init

In [15]:
!kubectl logs $pod_to_list

Using deprecated annotation `kubectl.kubernetes.io/default-logs-container` in pod/jmt-nginx-5bb4fb5ff4-7rb8f. Please use `kubectl.kubernetes.io/default-container` instead
/docker-entrypoint.sh: /docker-entrypoint.d/ is not empty, will attempt to perform configuration
/docker-entrypoint.sh: Looking for shell scripts in /docker-entrypoint.d/
/docker-entrypoint.sh: Launching /docker-entrypoint.d/10-listen-on-ipv6-by-default.sh
10-listen-on-ipv6-by-default.sh: info: Getting the checksum of /etc/nginx/conf.d/default.conf
10-listen-on-ipv6-by-default.sh: info: Enabled listen on IPv6 in /etc/nginx/conf.d/default.conf
/docker-entrypoint.sh: Launching /docker-entrypoint.d/20-envsubst-on-templates.sh
/docker-entrypoint.sh: Launching /docker-entrypoint.d/30-tune-worker-processes.sh
/docker-entrypoint.sh: Configuration complete; ready for start up
2021/12/19 17:22:53 [notice] 1#1: using the "epoll" event method
2021/12/19 17:22:53 [notice] 1#1: nginx/1.21.4
2021/12/19 17:22:53 [notice] 1#1: built 

In [16]:
!kubectl get service

NAME                              TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
ml-pipeline-visualizationserver   ClusterIP   10.43.199.188   <none>        8888/TCP   18h
ml-pipeline-ui-artifact           ClusterIP   10.43.212.116   <none>        80/TCP     18h
dask-development                  ClusterIP   10.43.200.151   <none>        80/TCP     18h
dask-testnotebook                 ClusterIP   10.43.241.104   <none>        80/TCP     18h
jmt-nginx                         ClusterIP   10.43.144.181   <none>        80/TCP     36s


In [17]:
!kubectl describe service jmt-nginx

Name:              jmt-nginx
Namespace:         kubeflow-user
Labels:            run=jmt-nginx
Annotations:       <none>
Selector:          run=jmt-nginx
Type:              ClusterIP
IP Families:       <none>
IP:                10.43.144.181
IPs:               10.43.144.181
Port:              <unset>  80/TCP
TargetPort:        80/TCP
Endpoints:         10.42.0.133:80,10.42.1.192:80
Session Affinity:  None
Events:            <none>


## Scale deployment


In [18]:
# get deployment object
nginx_deployment = v1_app_api.read_namespaced_deployment(
    'jmt-nginx',
    'kubeflow-user'

)
nginx_deployment.status.replicas, nginx_deployment.status.ready_replicas, \
    nginx_deployment.metadata.name, nginx_deployment.metadata.namespace

(2, 2, 'jmt-nginx', 'kubeflow-user')

### Scale up

In [19]:
# initialize metadata component
scaling_metadata = client.V1ObjectMeta(
    name=nginx_deployment.metadata.name,
    namespace=nginx_deployment.metadata.namespace
)
scaling_metadata

{'annotations': None,
 'cluster_name': None,
 'creation_timestamp': None,
 'deletion_grace_period_seconds': None,
 'deletion_timestamp': None,
 'finalizers': None,
 'generate_name': None,
 'generation': None,
 'labels': None,
 'managed_fields': None,
 'name': 'jmt-nginx',
 'namespace': 'kubeflow-user',
 'owner_references': None,
 'resource_version': None,
 'self_link': None,
 'uid': None}

In [20]:
# increase replicas to 3
scaling_spec = client.V1ScaleSpec(replicas=3)
scaling_spec

{'replicas': 3}

In [21]:
# scale up to 3 replicas
v1_app_api.replace_namespaced_deployment_scale(
    name=nginx_deployment.metadata.name,
    namespace=nginx_deployment.metadata.namespace,
    body=client.V1Scale(metadata=scaling_metadata, spec=scaling_spec),
    pretty='true'
)

{'api_version': 'autoscaling/v1',
 'kind': 'Scale',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2021, 12, 19, 17, 22, 48, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'jmt-nginx',
              'namespace': 'kubeflow-user',
              'owner_references': None,
              'resource_version': '589493',
              'self_link': None,
              'uid': '350c39bf-6d3a-49fd-89d3-78281a4b05fe'},
 'spec': {'replicas': 3},
 'status': {'replicas': 2, 'selector': 'run=jmt-nginx'}}

In [23]:
!kubectl get deploy jmt-nginx

NAME        READY   UP-TO-DATE   AVAILABLE   AGE
jmt-nginx   3/3     3            3           110s


### Scale Down

In [24]:
# scale down to 1 replica
scaling_spec = client.V1ScaleSpec(replicas=1)
v1_app_api.replace_namespaced_deployment_scale(
    name=nginx_deployment.metadata.name,
    namespace=nginx_deployment.metadata.namespace,
    body=client.V1Scale(metadata=scaling_metadata, spec=scaling_spec),
    pretty='true'
)

{'api_version': 'autoscaling/v1',
 'kind': 'Scale',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2021, 12, 19, 17, 22, 48, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'jmt-nginx',
              'namespace': 'kubeflow-user',
              'owner_references': None,
              'resource_version': '589911',
              'self_link': None,
              'uid': '350c39bf-6d3a-49fd-89d3-78281a4b05fe'},
 'spec': {'replicas': 1},
 'status': {'replicas': 3, 'selector': 'run=jmt-nginx'}}

In [25]:
!kubectl get deploy jmt-nginx

NAME        READY   UP-TO-DATE   AVAILABLE   AGE
jmt-nginx   1/1     1            1           2m14s


## clean up

In [26]:
!kubectl delete deploy jmt-nginx

deployment.apps "jmt-nginx" deleted


In [27]:
!kubectl delete service jmt-nginx

service "jmt-nginx" deleted
